<a href="https://colab.research.google.com/github/mohammadreza-mohammadi94/Deep-Learning-Projects/blob/main/Traffic_Forecasting_PeMSD7/Traffic_forecasting_PeMDS7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU, Dense, RepeatVector, TimeDistributed
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from typing import Tuple

# Configuration and Constants

In [ ]:
class Config:
    FILE_PATH = "PeMSD7_V_228.csv"
    INPUT_WINDOW = 12
    OUTPUT_HORIZON = 12
    SENSOR_TO_PROCESS = 0  # Index of the sensor column to use
    TRAIN_RATIO = 0.7
    VALIDATION_RATIO = 0.1
    UNITS = 256
    EPOCHS = 25
    BATCH_SIZE = 64
    PATIENCE = 10
    SENSOR_TO_PLOT = 0  # Since we use one sensor, this is always 0


In [ ]:
# 1. Check if GPU is available and properly detected
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Is Built with CUDA: ", tf.test.is_built_with_cuda())

# 2. Verify GPU is being used by the session
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Optional: Configure GPU memory growth to prevent OOM errors
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU setup is OK.")
    except RuntimeError as e:
        print(f"GPU setup error: {e}")
else:
    print("No GPU found.")

Num GPUs Available:  0
Is Built with CUDA:  True
No GPU found.


# Load Traffic Data

In [ ]:
!wget https://raw.githubusercontent.com/VeritasYin/STGCN_IJCAI-18/refs/heads/master/dataset/PeMSD7_Full.zip
!unzip PeMSD7_Full.zip

--2025-10-29 20:12:18--  https://raw.githubusercontent.com/VeritasYin/STGCN_IJCAI-18/refs/heads/master/dataset/PeMSD7_Full.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29211203 (28M) [application/zip]
Saving to: ‘PeMSD7_Full.zip.2’

PeMSD7_Full.zip.2   100%[===================>]  27.86M  --.-KB/s    in 0.1s    

2025-10-29 20:12:19 (237 MB/s) - ‘PeMSD7_Full.zip.2’ saved [29211203/29211203]

Archive:  PeMSD7_Full.zip
replace PeMSD7_V_228.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: PeMSD7_V_228.csv        
  inflating: PeMSD7_V_1026.csv       
  inflating: PeMSD7_W_228.csv        
  inflating: PeMSD7_W_1026.csv       


In [ ]:
def load_traffic_data(file_path: str) -> np.ndarray:
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: '{file_path}'.")
    df = pd.read_csv(file_path, header=None)  # The provided CSV has no header
    return df.values

# Create `X`, `y`

In [ ]:
def create_sliding_windows(data: np.ndarray, input_window: int, output_horizon: int) -> Tuple[np.ndarray, np.ndarray]:
    X, y = [], []
    total_timesteps = data.shape[0]
    for i in range(total_timesteps - input_window - output_horizon + 1):
        input_w = data[i: i + input_window, :]
        output_h = data[i + input_window : i + input_window + output_horizon, :]
        X.append(input_w)
        y.append(output_h)
    return np.array(X), np.array(y)

# Seq2Seq Model

In [ ]:
def build_seq2seq_model(input_shape: tuple, output_horizon: int, num_sensors: int, units: int) -> Model:
    """
    Builds a GRU-based Seq2Seq model with the corrected Functional API pattern.
    """
    # === ENCODER ===
    encoder_inputs = Input(shape=input_shape, name="encoder_input")

    # The first GRU layer's output is a single tensor.
    encoder_gru1_outputs = GRU(units, return_sequences=True, name="encoder_gru1")(encoder_inputs)

    # **THE FIX IS HERE:** Call the second GRU and unpack its two outputs on the same line.
    # We don't need the sequence output from this layer, only the state.
    _, encoder_state = GRU(units, return_state=True, name="encoder_gru2")(encoder_gru1_outputs)

    # === DECODER ===
    decoder_inputs = RepeatVector(output_horizon, name='repeat_vector')(encoder_state)

    # We apply the same direct pattern here.
    decoder_gru1_outputs = GRU(units, return_sequences=True, name='decoder_gru1')(decoder_inputs, initial_state=encoder_state)
    decoder_gru2_outputs = GRU(units, return_sequences=True, name='decoder_gru2')(decoder_gru1_outputs)

    # === OUTPUT LAYER ===
    output_layer = TimeDistributed(Dense(num_sensors), name="output_layer")
    outputs = output_layer(decoder_gru2_outputs)

    # Create and compile the final model
    model = Model(encoder_inputs, outputs)
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])
    return model

# Visualization

In [ ]:
def plot_prediction(y_true: np.ndarray, y_pred: np.ndarray, config: Config):
    plt.figure(figsize=(15, 6))
    sensor_idx = config.SENSOR_TO_PLOT
    true_values = y_true[:, sensor_idx]
    predicted_values = y_pred[:, sensor_idx]
    plt.plot(true_values, label='Ground Truth', color='blue', marker='o')
    plt.plot(predicted_values, label='Prediction', color='red', linestyle='--', marker='x')
    plt.title(f'Traffic Flow Prediction for Sensor #{sensor_idx}')
    plt.xlabel(f'Time Step (in {config.OUTPUT_HORIZON*5}-min horizon)')
    plt.ylabel('Traffic Flow')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
def main():
    config = Config()

    print(">>> Loading Data")
    traffic_data_raw = load_traffic_data(config.FILE_PATH)
    # BUG FIX: Select the sensor column while keeping it a 2D array.
    # [:, config.SENSOR_TO_PROCESS] -> returns 1D array (WRONG)
    # [:, [config.SENSOR_TO_PROCESS]] -> returns 2D array (CORRECT)
    traffic_features = traffic_data_raw[:, [config.SENSOR_TO_PROCESS]]

    print(">>> Preprocessing Data")
    total_timesteps = traffic_features.shape[0]
    train_end_idx = int(total_timesteps * config.TRAIN_RATIO)
    val_end_idx = train_end_idx + int(total_timesteps * config.VALIDATION_RATIO)

    train_data = traffic_features[:train_end_idx]
    val_data = traffic_features[train_end_idx:val_end_idx]
    test_data = traffic_features[val_end_idx:]

    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(train_data)
    val_scaled = scaler.transform(val_data)
    test_scaled = scaler.transform(test_data)

    X_train, y_train = create_sliding_windows(train_scaled, config.INPUT_WINDOW, config.OUTPUT_HORIZON)
    X_val, y_val = create_sliding_windows(val_scaled, config.INPUT_WINDOW, config.OUTPUT_HORIZON)
    X_test, y_test = create_sliding_windows(test_scaled, config.INPUT_WINDOW, config.OUTPUT_HORIZON)

    # Check if any data was generated. If windows are too large, these might be empty.
    if len(X_train) == 0 or len(X_val) == 0:
        raise ValueError("Not enough data to create training/validation splits with the given window sizes.")

    print(f"Training samples: X={X_train.shape}, y={y_train.shape}")
    print(f"Validation samples: X={X_val.shape}, y={y_val.shape}")
    print(f"Test samples: X={X_test.shape}, y={y_test.shape}")

    print(">>> Building Model")
    num_sensors = X_train.shape[2]
    model = build_seq2seq_model((config.INPUT_WINDOW, num_sensors), config.OUTPUT_HORIZON, num_sensors, config.UNITS)
    model.summary()

    print(">>> Training Model")
    checkpoint = ModelCheckpoint('best_traffic_model.keras', save_best_only=True, monitor='val_loss', mode='min')
    early_stopping = EarlyStopping(monitor='val_loss', patience=config.PATIENCE, restore_best_weights=True)

    history = model.fit(
        X_train, y_train,
        epochs=config.EPOCHS,
        batch_size=config.BATCH_SIZE,
        validation_data=(X_val, y_val),
        callbacks=[checkpoint, early_stopping],
        # Drop last batch if its size is not equal to BATCH_SIZE
        # This is not strictly necessary anymore as we removed batch_shape, but it's good practice.
    )

    print(">>> Evaluating Model")
    test_loss, test_mae = model.evaluate(X_test, y_test, batch_size=config.BATCH_SIZE, drop_remainder=True)
    print(f"Test Loss (MSE): {test_loss:.4f}")
    print(f"Test Mean Absolute Error: {test_mae:.4f}")

    print(">>> Making Predictions")
    sample_idx = 0
    # Ensure test set has enough samples for a full batch for prediction if needed, or handle single prediction
    if len(X_test) > 0:
        input_sample = X_test[sample_idx:sample_idx+1]
        ground_truth = y_test[sample_idx]

        prediction_scaled = model.predict(input_sample)[0]

        prediction_real = scaler.inverse_transform(prediction_scaled)
        ground_truth_real = scaler.inverse_transform(ground_truth)

        plot_prediction(ground_truth_real, prediction_real, config)
    else:
        print("Not enough test data to make a prediction.")
main()

>>> Loading Data
>>> Preprocessing Data
Training samples: X=(8847, 12, 1), y=(8847, 12, 1)
Validation samples: X=(1244, 12, 1), y=(1244, 12, 1)
Test samples: X=(2512, 12, 1), y=(2512, 12, 1)
>>> Building Model


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_input       │ (None, 12, 1)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_gru1 (GRU)  │ (None, 12, 256)   │    198,912 │ encoder_input[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_gru2 (GRU)  │ [(None, 256),     │    394,752 │ encoder_gru1[0][… │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ repeat_vector       │ (None, 12, 256)   │          0 │ encoder_gru2[0][… │
│ (RepeatVector)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_gru1 (GRU)  │ (None, 12, 256)   │    394,752 │ repeat_vector[0]… │
│                     │                   │            │ encoder_gru2[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_gru2 (GRU)  │ (None, 12, 256)   │    394,752 │ decoder_gru1[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output_layer        │ (None, 12, 1)     │        257 │ decoder_gru2[0][… │
│ (TimeDistributed)   │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,383,425 (5.28 MB)

 Trainable params: 1,383,425 (5.28 MB)

 Non-trainable params: 0 (0.00 B)

>>> Training Model
Epoch 1/25
139/139 ━━━━━━━━━━━━━━━━━━━━ 53s 316ms/step - loss: 0.0547 - mean_absolute_error: 0.1566 - val_loss: 0.0256 - val_mean_absolute_error: 0.1271
Epoch 2/25
139/139 ━━━━━━━━━━━━━━━━━━━━ 44s 316ms/step - loss: 0.0181 - mean_absolute_error: 0.0899 - val_loss: 0.0180 - val_mean_absolute_error: 0.0838
Epoch 3/25
139/139 ━━━━━━━━━━━━━━━━━━━━ 44s 320ms/step - loss: 0.0164 - mean_absolute_error: 0.0794 - val_loss: 0.0178 - val_mean_absolute_error: 0.0895
Epoch 4/25
139/139 ━━━━━━━━━━━━━━━━━━━━ 43s 307ms/step - loss: 0.0160 - mean_absolute_error: 0.0801 - val_loss: 0.0192 - val_mean_absolute_error: 0.0905
Epoch 5/25
139/139 ━━━━━━━━━━━━━━━━━━━━ 43s 309ms/step - loss: 0.0165 - mean_absolute_error: 0.0806 - val_loss: 0.0177 - val_mean_absolute_error: 0.0839
Epoch 6/25
139/139 ━━━━━━━━━━━━━━━━━━━━ 44s 315ms/step - loss: 0.0159 - mean_absolute_error: 0.0798 - val_loss: 0.0193 - val_mean_absolute_error: 0.1019
Epoch 7/25
139/139 ━━━━━━━━━━━━━━━━━━━━ 81s 308ms/step - loss: 

ValueError: Arguments not recognized: {'drop_remainder': True}